In [ ]:
from cortexlib.mouse import CortexlabMouse

mouse = CortexlabMouse()

null_srv_all_neurons = mouse.compute_null_all_neurons(n_shuffles=100)
real_srv_all_neurons = mouse.compute_real_srv_all_neurons()
reliable_neuron_indices = mouse.get_reliable_neuron_indices(
            null_srv_all_neurons, real_srv_all_neurons, percentile_threshold=99)
neural_responses_mean, _ = mouse.get_responses_for_reliable_neurons(reliable_neuron_indices, real_srv_all_neurons, num_neurons=500)

mouse.plot_null_distribution_for_neuron(null_srv_all_neurons, neuron_index=0)
mouse.plot_real_srv_distribution(real_srv_all_neurons, reliable_neuron_indices)

In [ ]:
from cortexlib.images import CortexlabImages

images = CortexlabImages()
images.plot_raw_image(int(mouse.stimulus_ids[0]))
image_dataset = images.load_images_shown_to_mouse(mouse.stimulus_ids)
images.show_sample(image_dataset, n=5)

In [ ]:
from cortexlib.simclr import PreTrainedModel

simclr = PreTrainedModel()
simclr_features = simclr.extract_features(image_dataset)

In [ ]:
### Compute RDMs and Spearman correlation between raw feats and neural responses
# Compares the structure of representational geometry
# Each RDM is a [n_images × n_images] matrix showing pairwise dissimilarities
# “Do SimCLR and mouse visual cortex organise the relationships between images in a similar way?”

from scipy.spatial.distance import pdist, squareform
from scipy.stats import spearmanr
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

def compute_rdm(X, metric='correlation'):
    return squareform(pdist(X, metric=metric))

# Flatten upper triangle to get vector form
def vectorize_rdm(rdm):
    triu_idx = np.triu_indices(rdm.shape[0], k=1)
    return rdm[triu_idx]

def compute_layerwise_rsa(simclr_features, neural_data):
    for layer, feats in simclr_features.items():
        if layer == 'labels':
            continue
        
        print(f"Layer: {layer}, Shape: {feats.shape}")

        if feats.dim() == 4: # all except fc, which SimCLR already flattens
            feats = feats.view(feats.size(0), -1)

        rdm_simclr = compute_rdm(feats)
        rdm_neural = compute_rdm(neural_data)

        sim = spearmanr(vectorize_rdm(rdm_neural), vectorize_rdm(rdm_simclr)).correlation

        print(f"Spearman correlation between neural RDM and SimCLR {layer} RDM: {sim:.4f}")

        sns.heatmap(rdm_neural)
        plt.title("Neural RDM")
        plt.show()

        sns.heatmap(rdm_simclr)
        plt.title("SimCLR Layer RDM")
        plt.show()

compute_layerwise_rsa(simclr_features, neural_responses_mean)

In [ ]:
### Same as above, but run PCA on the SimCLR features first

from scipy.spatial.distance import pdist, squareform
from scipy.stats import spearmanr
import numpy as np
from sklearn.decomposition import PCA

def compute_rdm(X, metric='correlation'):
    return squareform(pdist(X, metric=metric))

def vectorize_rdm(rdm):
    triu_idx = np.triu_indices(rdm.shape[0], k=1)
    return rdm[triu_idx]

# Compute neural RDM once
rdm_neural = compute_rdm(neural_responses_mean)
vec_neural = vectorize_rdm(rdm_neural)

# PCA settings
pca_list = [10, 50, 100, 200]

for layer, feats in simclr_features.items():
    if layer == 'labels':
        continue

    print(f"\nLayer: {layer}, Shape: {simclr_features[layer].shape}")

    if feats.dim() == 4:
        feats = feats.view(feats.size(0), -1)
    feats = feats.numpy()

    for n_components in pca_list:
        if feats.shape[1] < n_components:
            print(f"  Skipped PCA {n_components} (too many components for dim {feats.shape[1]})")
            continue

        pca = PCA(n_components=n_components)
        feats_pca = pca.fit_transform(feats)

        rdm_simclr = compute_rdm(feats_pca)
        vec_simclr = vectorize_rdm(rdm_simclr)

        sim = spearmanr(vec_neural, vec_simclr).correlation
        print(f"  PCA {n_components} → RSA: {sim:.4f}")

In [ ]:
### Compute RDMs and Spearman correlation between PC1 of neural responses and SimCLR features
# Compares the activation profile of each unit with the dominant axis of variation in the neural data
# “Is any SimCLR unit tracking what neural PC1 represents?”

pca = PCA(10)
neural_data_pcs = pca.fit_transform(neural_responses_mean)
pc1_neural_data = neural_data_pcs[:, 0]

results = []

for layer, feats in simclr_features.items():
    if layer == 'labels':
        continue

    print(f"\nLayer: {layer}, Shape: {feats.shape}")

    # Flatten spatial dims if needed
    if feats.dim() == 4:
        feats = feats.view(feats.size(0), -1)  # [1573, D]

    feats_np = feats.numpy()  # convert to numpy

    # Compute Spearman correlation between each feature dim and neural PC1
    corrs = []
    for i in range(feats_np.shape[1]):
        r, _ = spearmanr(feats_np[:, i], pc1_neural_data)
        corrs.append(r)

    # Record max correlation
    max_corr = np.max(np.abs(corrs))
    mean_corr = np.mean(np.abs(corrs))

    results.append((layer, max_corr))

    print(f"Max |Spearman| corr with neural PC1: {max_corr:.4f}")
    print(f"Mean |Spearman| corr: {mean_corr:.4f}")